# 1. env build

In [ ]:
!atp-get update
!apt-get upgrade

# 
# ---------------------------------------------------------------------------- #
!python3.7 -m pip install nvidia-ml-py3
!python3.7 -m pip install fastai==1.0.61
!python3.7 -m pip install lpips==0.1.3
!sudo apt-get install libsvm-dev
!python3.7 -m pip install pybrisque
!python3.7 -m pip install sk-video
!python3.7 -m pip install geffnet
# ---------------------------------------------------------------------------- #

!pip install "torch==1.4" "torchvision==0.5.0"
!pip install wandb
!pip install wandb --upgrade


# google permission authentication
# ---------------------------------------------------------------------------- #
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
 
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
# ---------------------------------------------------------------------------- #


# mount google cloud storage
# ---------------------------------------------------------------------------- #
# -----------------
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')
# ---------------------------------------------------------------------------- #

# base include
# ---------------------------------------------------------------------------- #
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
# ---------------------------------------------------------------------------- #

# ---------------------------------------------------------------------------- #
%cd /content/
!wget https://github.com/cjlin1/libsvm/archive/refs/tags/v324.tar.gz
!tar zxvf v324.tar.gz
%cd libsvm-324/
!make clean
!make
!rm /content/v324.tar.gz
# ---------------------------------------------------------------------------- #

# ---------------------------------------------------------------------------- #
# source code in github main
%cd /content/
!cp -R /content/drive/MyDrive/ARGAN-Temp/Artifact_Removal_GAN/ /content/
if not 'Artifact_Removal_GAN' in sys.path:
  sys.path += ['Artifact_Removal_GAN']
%cd Artifact_Removal_GAN/
!mkdir models
!wget -P /content/Artifact_Removal_GAN/models/ https://github.com/mameli/Artifact_Removal_GAN/releases/download/1.1/standard.pkl
# ---------------------------------------------------------------------------- #

In [ ]:
!wandb login --relogin

In [ ]:
wandbCallbacks = True
import datetime

if wandbCallbacks:
    import wandb
    from wandb.fastai import WandbCallback
    config={"batch_size": bs,
            "img_size": (sz, sz),
            "learning_rate": lr,
            "weight_decay": wd,
            "num_epochs": epochs
    }
    # wandb.init(project='SuperRes', config=config, id="unet_superRes_mobilenetV3_FID"+ datetime.datetime.now().strftime('_%m-%d_%H:%M'))
    wandb.init(settings=wandb.Settings(start_method='thread'))
    learn_gen.callback_fns.append(partial(WandbCallback, input_type='images'))

# 2.  Train Code

In [ ]:
path = Path('/content/drive/MyDrive/')

path_fullRes = path/'MyTrain444/'
path_lowRes_Full = path/'MyTrain420/'
proj_id = 'unet_superRes_mobilenetV3_LPIPS'

gen_name = proj_id + '_gen'
crit_name = proj_id + '_crit'

nf_factor = 2
pct_start = 1e-8

In [ ]:
bs=10
sz=256
lr = 1e-2
wd = 1e-3
epochs = 10

In [ ]:
import sys
sys.path.append('/content/libsvm-324/python')
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

from fastai import *
from fastai.vision import *
from fastai.vision.gan import *
from ArNet.generators import *
from ArNet.critics import *
from ArNet.dataset import *
from ArNet.loss import *
from ArNet.save import *
from ArNet.fid_loss import *
from ArNet.ssim import *
from ArNet.metrics import *

import torchvision
import geffnet # efficient/ mobile net

def do_fit(learn, epochs,save_name, lrs=slice(1e-3), pct_start=0.9):
    learn.fit_one_cycle(epochs, lrs, pct_start=pct_start)
    learn.save(save_name)
    learn.show_results(rows=1, imgsize=20)

In [ ]:
model = geffnet.mobilenetv3_rw

loss_func = lpips_loss()
loss_func=loss_func.cuda()

data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=bs, sz=sz)

learn_gen = gen_learner_wide(data=data_gen,
                             gen_loss=loss_func,
                             arch = model,
                             nf_factor=nf_factor)

metrics append

In [ ]:
learn_gen.metrics.append(SSIM_Metric_gen())
learn_gen.metrics.append(SSIM_Metric_input())
learn_gen.metrics.append(LPIPS_Metric_gen())
learn_gen.metrics.append(LPIPS_Metric_input())
learn_gen.metrics.append(BRISQUE_Metric_gen())
learn_gen.metrics.append(BRISQUE_Metric_input())
learn_gen.metrics.append(BRISQUE_Metric_target())
learn_gen.metrics.append(NIQE_Metric_gen())
learn_gen.metrics.append(NIQE_Metric_input())
learn_gen.metrics.append(NIQE_Metric_target())

learning rate 

In [ ]:
learn_gen.lr_find()
learn_gen.recorder.plot()
learn_gen.summary()

In [ ]:
# save path ...
do_fit(learn_gen, 25, gen_name+"_256px_0", slice(lr*10))
# learn_gen.unfreeze()
# do_fit(learn_gen, 1, gen_name+"_256px_1", lr)